In [1]:
import sys
sys.path.append('../CrawlData')
from kafka import KafkaProducer
import requests
from json import dumps
import time
from kafka import KafkaConsumer

In [2]:
from GetItemDetail import getItemDetail
from GetItemShopID import getItemShopID

In [3]:
import pandas as pd
categoryList = pd.read_csv('../CrawlData/Data/categoryList.csv')

In [1]:
import json
from urllib.request import urlopen, Request


def get_comment(ItemListID, timeout=10):
    features = ['orderid', 'itemid', 'cmtid', 'ctime', 'rating', 'userid', 'shopid', 
            'comment', 'rating_star', 'status', 'mtime', 'editable', 'opt', 'filter']

    url = 'https://shopee.vn/api/v2/item/get_ratings?filter=0&flag=0&itemid={}&limit=6&offset={}&shopid={}&type=0'
    
    out_put_data = []
    
    for itemid, shopid in ItemListID:
        try:
            rep = Request(url.format(itemid, 0, shopid), headers={'User-Agent': 'Mozilla/5.0'})
            page = urlopen(rep, timeout=timeout).read()
            data = json.loads(page)
            rating_total = data['data']['item_rating_summary']['rating_total']
        except:
            print('exception requests: ', itemid, shopid)
            continue


        for offset in range(0, rating_total, 6):

            try:
                rep = Request(url.format(itemid, offset, shopid), headers={'User-Agent': 'Mozilla/5.0'})
                page = urlopen(rep, timeout=timeout).read()
                data = json.loads(page)

                for cmt in data['data']['ratings']:
                    comment = {}
                    for ft in features:
                        comment[ft] = cmt[ft]
                    out_put_data.append(comment)

            except:
                print("exception:", itemid, offset, shopid)

    
    return out_put_data


In [2]:
test = get_comment([[6758420150, 187524700]])

exception: 6758420150 6 187524700
exception: 6758420150 18 187524700
exception: 6758420150 36 187524700
exception: 6758420150 54 187524700
exception: 6758420150 60 187524700
exception: 6758420150 72 187524700
exception: 6758420150 78 187524700
exception: 6758420150 90 187524700


In [ ]:
test

[{'orderid': 89599412167890,
  'itemid': 6758420150,
  'cmtid': 6540289731,
  'ctime': 1636537314,
  'rating': 1,
  'userid': 178212802,
  'shopid': 187524700,
  'comment': 'Hàng đẹp giao hàng nhanh sẽ ủng hộ shop nhiều ak...!?!?!!_!!',
  'rating_star': 5,
  'status': 2,
  'mtime': 1636537314,
  'editable': 0,
  'opt': 2,
  'filter': 7},
 {'orderid': 84214867281781,
  'itemid': 6758420150,
  'cmtid': 5953429596,
  'ctime': 1631416914,
  'rating': 1,
  'userid': 20878112,
  'shopid': 187524700,
  'comment': 'Sản phẩm rất xinh đẹp tuyệt vời, tôi không thể trông đợi gì hơn. Cảm ơn nhà cung cấp, nhà sản xuất, cảm ơn người bán hàng, cảm ơn sàn thương mại điện tử, cảm ơn người vận chuyển để tôi có cơ hội dùng sản phẩm tốt như thế này.',
  'rating_star': 5,
  'status': 2,
  'mtime': 1631416914,
  'editable': 0,
  'opt': 2,
  'filter': 7},
 {'orderid': 83261280685886,
  'itemid': 6758420150,
  'cmtid': 5858090622,
  'ctime': 1630559821,
  'rating': 1,
  'userid': 10583121,
  'shopid': 18752470

In [5]:
# from a - b category
a = 0
b = 1
ItemListID = 0
for item in range(a,b):
    topic = categoryList.catid.iloc[item]
    print(f'topic {item} "{categoryList.display_name.iloc[item]}" is runing...')
    for newest in range(0,8001,60): 
        ItemListID = getItemShopID(topic,newest)
        break
    break

topic 0 "Thời Trang Nam" is runing...


In [14]:
# from a - b category
a = 0
b = 1
for item in range(a,b):
    topic = categoryList.catid.iloc[item]
    print(f'topic {item} "{categoryList.display_name.iloc[item]}" is runing...')
    for newest in range(0,8001,60): 
        ItemListID = getItemShopID(topic,newest)
        data = getItemDetail(ItemListID,topic)
        break
    break

topic 0 "Thời Trang Nam" is runing...


 13%|█▎        | 8/60 [02:23<37:01, 42.73s/it]

Crawl website error


 15%|█▌        | 9/60 [04:34<59:47, 70.35s/it]

Crawl website error


 22%|██▏       | 13/60 [06:28<39:29, 50.41s/it]

Crawl website error


 45%|████▌     | 27/60 [06:43<00:39,  1.21s/it]

In [ ]:
df=pd.DataFrame(data)

In [9]:
from kafka import KafkaProducer
import requests
from json import dumps
import time
from kafka import KafkaConsumer

In [17]:
producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [20]:
newDict = {}
newDict['data'] = data

In [ ]:
for item in data:
    print(item)
    producer.send('test', value=item)

In [21]:
producer.send('test', value=newDict)

TypeError: Object of type int64 is not JSON serializable